My comments may seem a bit verbose because the intention is for the next intern to understand where I left off. Of course do your own research as I could be wring and there are probably better ways of doing this that I don't know.

In [7]:
#Load the libraries
# You may need to use conda in instead of pip to install torch and by extension the rest of these. A second 
# A second issue that you might have is trying to do any sort of deep learning like what you're doing here
# without a GPU, namely one that has CUDA. You can still do it this way using your CPU but it'll run REALLY slow
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import timm
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


The goal here is to use geemap (gee stands for Google Earth Engine) to get satellite images of the roofs. But to be honest, if you can find a good dataset with ALOT of rooftop images, use that instead.

In [2]:
import geemap
import ee

# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project= "ee-loisaidaalbedo24")


# Create a map
Map = geemap.Map()

# Example: Define an area of interest by coordinates
aoi = ee.Geometry.Polygon([[
[40.7661, -73.9876],
[40.717087, -73.984536],
[40.713645, -73.988087],
[40.710536, -73.991699],
[40.708964, -73.991834],
[40.706721, -73.987287]
  ]])

# Create a map centered on the AOI
Map = geemap.Map(center=[40.748, -73.985], zoom=12)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 3000}, 'Satellite Image')
Map.addLayer(aoi, {}, 'AOI')

# Display the map
Map


# Get a satellite image collection
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(aoi) \
    .sort('CLOUD_COVER') \
    .limit(10)

# Select the image band for visualization, e.g., True Color
image = collection.median().clip(aoi)






In [ ]:
Map = geemap.Map(center=[40.748, -73.985], zoom=12)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 3000}, 'Satellite Image')
Map.addLayer(aoi, {}, 'AOI')

Map


In [ ]:
class RooftopDataset(Dataset):
    def __init__(self, data_dir, transform = None):
        self.data = ImageFolder(data_dir, transform=transform)
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return self.data[idx]
    def classes(self):
        return self.data.classes


In [8]:
dataset = RooftopDataset(data_dir = )

#Use Kyle's code to get an approximate albedo value for the labels. This is a crude estimator but, it's better than nothing.



SyntaxError: expected argument value expression (2725475908.py, line 1)

In [5]:
#Make a dictionary that matches the images to their classifications
data_dir = #Dataset               

pic_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(pic_to_class)

#Normalize and make the images into tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


Construct the CNN, look it up so you know how it works.

In [10]:


class AlbedoEstimator(nn.Module):
    def __init__(self, num_colors = 2):
        super(AlbedoEstimator,self).__init__()
        self.base_model = timm.create_model('efficientnet_b0',pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        
        enet_out_size = 1280

        self.estimator = nn.Linear(enet_out_size,num_colors)
    
    def forward(self, x):
       # Pass input through the feature extractor
        x = self.features(x)
        # Flatten the output of the feature extractor
        x = x.view(x.size(0), -1)
        # Pass the flattened output through the linear layer
        x = self.estimator(x)
        return x 





In [ ]:
model = AlbedoEstimator(num_colors = 2)
print(model)

#Use softmax for the prob. estimation layer:
softmax = nn.Softmax(dim=1)


Model Training and Validation

In [ ]:
train_folder = #insert file path here

test_folder = #insert file path here

val_folder = #insert file path here

train_data = RooftopDataset(train_folder,transform = transform)

test_data = RooftopDataset(test_folder,transform = transform)

val_data = RooftopDataset(val_folder,transform = transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
num_epochs = 5
train_losses , val_losses = [],[]


device = torch.device("cpu")


model = AlbedoEstimator(num_colors=2)
model.to(device)

#Establish loss function
loss_function = nn.CrossEntropyLoss()

#Get an optimizer, for now I can just use ADAM 
optimizer = optim.Adam(model.parameters(), lr = 0.001)

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

num_epochs = 5
train_losses, val_losses = [], []

device = torch.device("cpu")

model = AlbedoEstimator(num_colors=2)
model.to(device)

# Establish loss function
loss_function = nn.CrossEntropyLoss()

# Get an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        classifications = model(images)
        loss = loss_function(classifications, labels)  
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            images, labels = images.to(device), labels.to(device)
         
            classifications = model(images)
            loss = loss_function(classifications, labels)  # Use raw logits here
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

# Get the probabilities for each classification. These will be interpreted as the albedo values
model.eval()
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        classifications = model(images)
        albedo = torch.softmax(classifications, dim=1) 
        
